In [216]:
import pandas as pd
from math import log2

pd.options.display.float_format = "{:,.0f}".format


## Chord System

In [220]:
class ChordSystem:
    def __init__(self, ring_size, active_nodes):
        self.ring_size = ring_size
        self.finger_table_size = int(log2(self.ring_size))
        self.active_nodes = active_nodes

    def get_succ(self, node):
        i = node
        while i not in self.active_nodes:
            i = (i + 1) % self.ring_size
        return i

    def get_succs(self, nodes=None):
        if not nodes:
            nodes = [i for i in range(self.ring_size)]
        succs = [pd.Series({"succ": self.get_succ(node)}, name=node) for node in nodes]
        return pd.DataFrame(succs)

    def get_finger_table(self, active_node):
        finger_table = {}
        for i in range(self.finger_table_size):
            finger_table[f"i_{i}"] = self.get_succ(active_node + 2**i)
        return pd.Series(finger_table, name=active_node)

    def get_finger_tables(self, nodes=None):
        if not nodes:
            nodes = self.ctive_nodes
        return pd.DataFrame([self.get_finger_table(node) for node in nodes])

    def get_path(self, start, end):
        path = f"{start}"
        current = start
        while True:
            if self.get_succ(end) == current:
                break
            finger_table = self.get_finger_table(current).values
            next = finger_table[0]
            for node in finger_table[1:]:
                if node > end:
                    break
                next = node
            current = next
            path += f"-{current}"
        return path


# creation

In [221]:
ring_size = 16
active_nodes = {2, 4, 7, 8, 10, 12, 15}
cs = ChordSystem(ring_size, active_nodes)


# succ

In [222]:
nodes_to_get_succ_for = {0, 2, 4, 6, 13, 15}
cs.get_succs(nodes_to_get_succ_for)


,succ
0,2
2,2
4,4
6,7
13,15
15,15


# finger table

In [225]:
cs.get_finger_tables({4, 8, 10, 11}).sort_index()


,i_0,i_1,i_2,i_3
4,7,7,8,12
8,10,10,12,2
10,12,12,15,4
11,12,15,15,4


# path

In [218]:
start = 4
end = 11
cs.get_path(start, end)


'4-8-10-12'